In [29]:
import numpy as np
import pandas as pd
#import matplotlib as plt
#import seaborn as sns
import os

In [30]:
df = pd.read_csv('Data/fashion.csv')

In [31]:
df.head()

,ProductId,Gender,Category,SubCategory,ProductType,Colour,Usage,ProductTitle,Image,ImageURL
0,42419,Girls,Apparel,Topwear,Tops,White,Casual,Gini and Jony Girls Knit White Top,42419.jpg,http://assets.myntassets.com/v1/images/style/p...
1,34009,Girls,Apparel,Topwear,Tops,Black,Casual,Gini and Jony Girls Black Top,34009.jpg,http://assets.myntassets.com/v1/images/style/p...
2,40143,Girls,Apparel,Topwear,Tops,Blue,Casual,Gini and Jony Girls Pretty Blossom Blue Top,40143.jpg,http://assets.myntassets.com/v1/images/style/p...
3,23623,Girls,Apparel,Topwear,Tops,Pink,Casual,Doodle Kids Girls Pink I love Shopping Top,23623.jpg,http://assets.myntassets.com/v1/images/style/p...
4,47154,Girls,Apparel,Bottomwear,Capris,Black,Casual,Gini and Jony Girls Black Capris,47154.jpg,http://assets.myntassets.com/v1/images/style/p...


In [32]:
df.shape

(2906, 10)

In [33]:
df.isnull().sum()

ProductId       0
Gender          0
Category        0
SubCategory     0
ProductType     0
Colour          0
Usage           0
ProductTitle    0
Image           0
ImageURL        0
dtype: int64

In [34]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [35]:
#df['Category'] = df['Category'].apply(collapse)
#df['SubCategory'] = df['SubCategory'].apply(collapse)
#df['ProductType'] = df['ProductType'].apply(collapse)
#df['Colour'] = df['Colour'].apply(collapse)
#df['Usage'] = df['Usage'].apply(collapse)

In [36]:
df['tags'] = df['Category'] + df['SubCategory'] + df['ProductType'] + df['Colour'] + df['Usage']

In [37]:
df.head(2)

,ProductId,Gender,Category,SubCategory,ProductType,Colour,Usage,ProductTitle,Image,ImageURL,tags
0,42419,Girls,Apparel,Topwear,Tops,White,Casual,Gini and Jony Girls Knit White Top,42419.jpg,http://assets.myntassets.com/v1/images/style/p...,ApparelTopwearTopsWhiteCasual
1,34009,Girls,Apparel,Topwear,Tops,Black,Casual,Gini and Jony Girls Black Top,34009.jpg,http://assets.myntassets.com/v1/images/style/p...,ApparelTopwearTopsBlackCasual


In [38]:
df1 = df.drop(columns=['Category','SubCategory','ProductType','Colour','Usage','Gender','Image','ImageURL'])

In [48]:
#df2=df1
#df2['tags'] = df1['tags'].apply(lambda x:x.split())

In [47]:
#df2['tags'] = df2['tags'].apply(lambda x: " ".join(x))

In [40]:
df1.head(2)

,ProductId,ProductTitle,tags
0,42419,Gini and Jony Girls Knit White Top,ApparelTopwearTopsWhiteCasual
1,34009,Gini and Jony Girls Black Top,ApparelTopwearTopsBlackCasual


In [49]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [50]:
vector = cv.fit_transform(df1['tags']).toarray()

In [51]:
vector.shape

(2906, 326)

In [52]:
from sklearn.metrics.pairwise import cosine_similarity

In [54]:
similarity = cosine_similarity(vector)
similarity

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [63]:
def recommend(movie):
    index = df1[df1['ProductTitle'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(df1.iloc[i[0]])
        print(df1.iloc[i[0]].ProductTitle)
        

In [66]:
#recommend('Gini and Jony Girls Black Top')
#recommend('Do u speak Green Girls Olive 3/4 Pant')
recommend('Nike Men Zoom Hyperenforcer White Sports Shoes')

ProductId                                        6628
ProductTitle    ADIDAS Men's Lowell White Silver Shoe
tags             FootwearShoesSports ShoesWhiteSports
Name: 1336, dtype: object
ADIDAS Men's Lowell White Silver Shoe
ProductId                                         33822
ProductTitle    Puma Men Axis White & Grey Sports Shoes
tags               FootwearShoesSports ShoesWhiteSports
Name: 1341, dtype: object
Puma Men Axis White & Grey Sports Shoes
ProductId                                      42089
ProductTitle    ADIDAS Men White Corona Sports Shoes
tags            FootwearShoesSports ShoesWhiteSports
Name: 1344, dtype: object
ADIDAS Men White Corona Sports Shoes
ProductId                                                  42042
ProductTitle    Nike Men Air Relentless 2 MSL White Sports Shoes
tags                        FootwearShoesSports ShoesWhiteSports
Name: 1352, dtype: object
Nike Men Air Relentless 2 MSL White Sports Shoes
ProductId                                       

In [61]:
import pickle

In [62]:
pickle.dump(df1,open('Products.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))